## Data Cleaning

In [5]:
import pandas as pd 
from pandas.io.json import json_normalize
import numpy as np 
from pymongo import MongoClient
import json

### Database Connection

In [6]:
DB_USERNAME = 'anutibara'
DB_PASSWORD = 'anutibara'
DB_HOST = 'scraping-cluster-7dtgt.gcp.mongodb.net'
DB_NAME = 'scraping_db'
try:
    client = MongoClient(f'mongodb+srv://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}')
    print("Database connected successfully")
except Exception as e:
    print("Error to connect to database: ", e)
db = client.get_database('scraping_db')
properties = db.properties

Database connected successfully


### Database Queries

##### Total Documents

In [7]:
properties.count_documents({})

0

### Convert Mongo Collection to DataFrame

In [4]:
json_documents = list(properties.find({}))
documents_df = pd.DataFrame(json_documents)

### DataFrame Transformations

In [45]:
documents_df

,_id,code,status,type,use,modifyDate,location,propertyAgency,description,features,moreFeatures,nameProject,builderCompany,offersType
0,5da716ea127afa8822c5a0da,4871253,Active,Venta,Usado,10/10/2019 6:33:04 PM,"[{'country': 'Colombia', 'department': 'Antioq...","[{'id': 684002, 'name': 'Elite Inmobiliaria', ...","Se vende apartamento en Calasanz, medellin con...","[{'price': '$ 215.000.000', 'squareMeters': '6...","[{'interiorFeatures': ['Balcón', 'Baño Indepen...",NaN,NaN,NaN
1,5da716ea57f52a57bcc5a0da,2800362,Active,Venta,Nuevo,3/29/2019 9:38:04 PM,"[{'country': 'Colombia', 'department': 'Antioq...",NaN,CITTÉ se encuentra ubicado en la vía las Palma...,[{'price': 'Desde $ 395.632.500 Hasta $ 411.63...,"[{'interiorFeatures': ['Balcón', 'Estudio', 'G...",CITTÉ,"[{'id': 383498, 'name': 'Constructora Monserra...","[{'property': 'Apartamento', 'offerType': 'Ven..."
2,5da716eaa35351fa84c5a0da,4916815,Active,Venta,Usado,10/13/2019 11:01:33 AM,"[{'country': 'Colombia', 'department': 'Antioq...","[{'id': 345580, 'name': 'Prointegral', 'contra...",Apartamento en unidad cerrada sobre la loma de...,"[{'price': '$ 520.000.000', 'squareMeters': '1...","[{'interiorFeatures': ['Balcón', 'Calentador',...",NaN,NaN,NaN
3,5da716ea8e7d67910ac5a0da,2565842,Active,Venta,Nuevo,10/11/2019 7:24:17 PM,"[{'country': 'Colombia', 'department': 'Antioq...",NaN,"Aptos de 41 a 115 m2. De 1, 2 y 3 alcobas con ...",[{'price': 'Desde $ 314.610.582 Hasta $ 775.32...,"[{'interiorFeatures': ['Balcón', 'Cocina Integ...",LA RIVIERE,"[{'id': 28481, 'name': 'Coninsa Ramón H. S.A.'...","[{'property': 'Apartamento', 'offerType': 'Ven..."
4,5da716ea062b9d807bc5a0da,4636494,Active,Venta,Usado,10/3/2019 10:39:32 PM,"[{'country': 'Colombia', 'department': 'Antioq...","[{'id': 416189, 'name': 'Arrendamientos Alnago...","2 ALCOBA, SALA COMEDOR, 1 CLOSET, COCINA INTEG...","[{'price': '$ 190.000.000', 'squareMeters': '7...",,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55509,5da9ed24b999a9706020ec08,2800362,Active,Venta,Nuevo,3/29/2019 9:38:04 PM,"[{'country': 'Colombia', 'department': 'Antioq...",NaN,CITTÉ se encuentra ubicado en la vía las Palma...,[{'price': 'Desde $ 395.632.500 Hasta $ 411.63...,"[{'interiorFeatures': ['Balcón', 'Estudio', 'G...",CITTÉ,"[{'id': 383498, 'name': 'Constructora Monserra...","[{'property': 'Apartamento', 'offerType': 'Ven..."
55510,5da9ed2950081d4fbc20ec04,3855742,Active,Venta,Usado,8/12/2019 4:23:42 PM,"[{'country': 'Colombia', 'department': 'Antioq...","[{'id': 419247, 'name': 'Arrendamientos Del Ce...","ÁREA: 3340M2, 3 PLANTAS, CON MALACATE Y OFICIN...","[{'price': '$ 6.800.000.000', 'squareMeters': ...",,NaN,NaN,NaN
55511,5da9ed296ab406074220ec08,1785570,Active,Venta,Usado,3/18/2019 9:15:19 PM,"[{'country': 'Colombia', 'department': 'Antioq...","[{'id': 176092, 'name': 'Abad Faciolince S.A',...","Casa con buena ubicación, con espacios muy amp...","[{'price': '$ 8.500.000.000', 'squareMeters': ...","[{'interiorFeatures': ['Balcón', 'Baño de Serv...",NaN,NaN,NaN
55512,5da9ed3150081d4fbc20ec06,2565842,Active,Venta,Nuevo,10/18/2019 4:09:41 PM,"[{'country': 'Colombia', 'department': 'Antioq...",NaN,"Aptos de 41 a 115 m2. De 1, 2 y 3 alcobas con ...",[{'price': 'Desde $ 314.610.582 Hasta $ 775.32...,"[{'interiorFeatures': ['Balcón', 'Cocina Integ...",LA RIVIERE,"[{'id': 28481, 'name': 'Coninsa Ramón H. S.A.'...","[{'property': 'Apartamento', 'offerType': 'Ven..."
